# X-ray Auto Labeller and Crop

## Features: 
#### 1. Takes X-ray scan and:
- Automatically generate PascalVOC labels (saved in XML folder)
- Crop objects out from main image

#### 2. Takes any given PascalVOC label and:
- Generate cropped images from the labels provided

## Pre-requisites
- root folder containing:
    - folder of images to process
    - output folder

---

# 1. Cell below will run the labelling + crop process
### How to Use: 
- Set up directories to use under "FOR USER INPUT" section
- Comment the 2 blocks of code below out first and run all.
- Set CROP_IMAGES to True if you want cropped images and labels to be generated, else write False for just labels
- Set OBJECT_CLASS to the type of object you are identifying
- set CONTOUR_THRESHOLD for boxing. For light coloured backgrounds, use high number. Dark backgrounds use low number. Adjust to find best result
- Run block of code below
    - Check preview to see if you are satisfied with the sample bounding boxes
- **May need to modify some automatically generated boxes manually. Run next block of code to redo the cropping  from modified XML labels if needed**

In [ ]:
CROP_IMAGES = True
OBJECT_CLASS = 'AN'

# Threshold for contour finding (0-255)
CONTOUR_THRESHOLD = 245

_=run_whole_process()

## 2. Run this cell to crop with manually modified XML labels

In [ ]:
# # XML Labels Folder **relative to root**
# XML_LABELS_FOLDER = "datasets/annotations/myLabels_2021_07_27-14.12.10_manualedit"

# crop_from_manual_XML()

---
# **** Directories & Parameters - FOR USER INPUT ****

In [ ]:
import os

# Coordinates to identify X-ray region in image
XMIN, YMIN = (0,0)
XMAX, YMAX = (1280, 900)   

# Min and Max size of bounding box to record (in px)
# Assume images of size 1024px * 1280px
MIN_BBOX_SIZE = 30   # Making this too small may cause incorrect boxes to appear (noise)
MAX_BBOX_SIZE = 600  # Making this too large may cause the main window to be taken as a label too

PRINT_FREQ = 5       # Frequency of status updates

In [ ]:
# Specify root directory - all directories are specified relative to this!
root = 'home/your_root_here'

import os
os.chdir(root)
os.getcwd()

In [ ]:
# Input & Output Directories (Relative to ROOT)

image_dir = "ScreenRecord_131353031904_imgs"                   # Where the UNCROPPED images are stored
output_dir = "ScreenRecord_131353031904_imgs/output"           # Output directory for annotations and extracted samples (2 folders will be created)

---
# Import Libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
# from PIL import Image, ImageFilter, ImageDraw, ImageChops, ImageFont
import cv2
import xml.etree.ElementTree as ET
from xml.dom import minidom
from datetime import datetime

---
# No more editing from here on

## Runs entire process

In [ ]:
def run_whole_process():
    
    ### ====== SETUP ====== ###
    
    # Define full paths
    curr_time = generate_unique_name()
    image_dir_full = os.path.join(root, image_dir)
    output_dir_full = os.path.join(root, output_dir)
    annotations_dir_full = os.path.join(output_dir_full, 'annotations', f"myLabels_{curr_time}")
    cropped_dir_full = os.path.join(output_dir_full, 'cropped_samples', f"myCropImgs_{curr_time}")
    
    # Get full list of input images
    input_imgs = [name for name in os.listdir(image_dir_full) if os.path.isfile(os.path.join(image_dir_full, name))]
    
    # Let user inspect if boxes are ok, else exit. user_inspect_bbox() returns True if user is satisfied with the boxes.
    if not user_inspect_bbox(cv2.imread(os.path.join(image_dir_full, input_imgs[np.random.randint(low=0, high=len(input_imgs), size=1)[0]]))):
        print("\nAlright. Modify CONTOUR_THRESHOLD and/or other parameters and run this block of code again! \nTip: Increase 'MIN_BBOX_SIZE' if there are many incorrect small boxes.")
        return 0
    
    # Create output folders for labels & cropped images
    if not os.path.exists(os.path.join(output_dir_full, 'annotations')):
        os.makedirs(os.path.join(output_dir_full, 'annotations'))
    os.makedirs(annotations_dir_full)
        
    if CROP_IMAGES:
        print("\nGreat! Now generating XML annotations and cropping images for entire dataset...")
        if not os.path.exists(os.path.join(output_dir_full, 'cropped_samples')):
            os.makedirs(os.path.join(output_dir_full, 'cropped_samples'))
        os.makedirs(cropped_dir_full)
    else:
        print("\nGreat! Now generating XML annotations for entire dataset...")
    
    
    ### ====== Proceed to generate labels and crop images for full dataset ====== ###
    
    # Cycle thru all images in library
    for i, img_name in enumerate(input_imgs):
        
        # Get image name (without file ext)
        img_name_no_ext = os.path.splitext(img_name)[0]
        
        # Open image and get dimensions
        curr_img_full_dir = os.path.join(image_dir_full, img_name)
        curr_img = cv2.imread(curr_img_full_dir)
        img_height, img_width, _ = curr_img.shape
        
        # Create label XML backbone
        xml_annotation = create_xml_backbone(img_name, curr_img_full_dir, img_width, img_height)
        
        # Get all contours from one image
        ctours = get_all_ctours(curr_img)
        
        # Each contour_entry is a bbox
        counter=0
        for contour_entry in ctours:
            
            # Get bounding rectangle
            x,y,w,h = cv2.boundingRect(contour_entry)
        
            # Only record down bboxes that are in X-ray zone and is not to big nor small
            if YMIN<=y<=YMAX and XMIN<=x<=XMAX and pass_size_test(w,h):
                counter +=1
                
                # Generate label by editing XML file
                xml_annotation = update_bbox_in_XML(xml_annotation, img_name, (x,y), w, h)
            
                if CROP_IMAGES:
                    # Crop image to the current bounding box
                    create_one_crop(x,y,w,h, curr_img, img_name_no_ext, cropped_dir_full, counter)

        # Save annotation XML
        dom = minidom.parseString(ET.tostring(xml_annotation))
        xml_out = dom.toprettyxml(indent='\t')
        with open(os.path.join(annotations_dir_full, f"{img_name_no_ext}.xml"), "w") as f:
            f.write(xml_out)
        
        # Counter
        if (i+1)%PRINT_FREQ==0 or i==(len(input_imgs)-1):
            print(f"{i+1}/{len(input_imgs)} processed!")
          
    
    if CROP_IMAGES:
        print("\nSuccess! XML annotations generated and images cropped!")
    else:
        print("\nSuccess! XML annotations generated!")

---

## Helper Functions - Image Related

In [ ]:
def get_all_ctours(image):
    """
    Function that gets all primary contours of image (i.e., find objects in scan)
    
    Input:
        image (cv2 object)
        
    Output:
        ctours (list): List of contours
    """
    gray =cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _,binary = cv2.threshold(gray,CONTOUR_THRESHOLD,255,cv2.THRESH_BINARY)
    ctours,_ = cv2.findContours(binary,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    return ctours

In [ ]:
def user_inspect_bbox(test_img):
    """
    Function that creates prompt for user to determine if current threshold is satisfactory
    
    Input:
        test_img (cv2 object): Random image in directory
        
    Output:
        user_results (bool): True/False
    """
    # Generates image preview with bounding boxes for user.
    gray =cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    _,binary = cv2.threshold(gray,CONTOUR_THRESHOLD,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(binary,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    for contour_entry in contours:
        x,y,w,h = cv2.boundingRect(contour_entry)
        
        # Only plot bboxes that are in X-ray zone and is not to big nor small
        if YMIN<=y<=YMAX and XMIN<=x<=XMAX and pass_size_test(w,h):
            cv2.rectangle(test_img,(x,y),(x+w,y+h),(0,255,0),2)
    
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (100,100))
    plt.imshow(test_img)
    plt.title('Test Output')
    plt.show()
    
    # User to decide if bounding boxes are satisfactory.
    while True:
        user_input = input("Are you satisfied with these bounding boxes? (Y/N): ")

        if user_input in ['Y', 'y', 'Y ', 'y ', 'yes']:
            return True
        elif user_input in ['N', 'n', 'N ', 'n ', 'no']:
            return False
        else:
            print("Invalid answer. Try again.")

## Helper Functions - XML Related

In [ ]:
def create_xml_backbone(curr_filename, save_dir, img_width, img_height):
    """
    PascalVOC Format
    Creates backbone of XML label file, populating tree with everything except object bounding box labels

    Returns:
        1. xml_annotation (ElementTree Object)
    """
    # Main node
    xml_annotation = ET.Element("annotation")

    # Primary nodes
    ET.SubElement(xml_annotation, "folder").text = "images"
    ET.SubElement(xml_annotation, "filename").text = str(curr_filename)
    ET.SubElement(xml_annotation, "path").text = str(save_dir)
    source = ET.SubElement(xml_annotation, "source")
    size = ET.SubElement(xml_annotation, "size")
    ET.SubElement(xml_annotation, "segmented").text = "0"

    # Source child node
    ET.SubElement(source, "database").text = "Unknown"

    # Size child nodes
    ET.SubElement(size, "width").text = str(img_width)
    ET.SubElement(size, "height").text = str(img_height)
    ET.SubElement(size, "depth").text = "1"

    return xml_annotation

In [ ]:
def update_bbox_in_XML(xml_annotation, selected_threat, placement_coords,
                       threat_width, threat_height):
    """
    Adds bounding box info "Object Node" into XML labels file

    Inputs:
        1. xml_annotation (XML ElementTree Object)
        2. selected_threat (Name without file extension)
        3. placement_coords (Tuple in x,y format)
        4. threat_width (x)
        5. threat_height (y)

    Returns:
        1.  xml_annotation (XML ElementTree Object)
    """
    myObject = ET.SubElement(xml_annotation, "object")

    # Object child nodes
    ET.SubElement(myObject, "name").text = str(OBJECT_CLASS)
    ET.SubElement(myObject, "pose").text = "Unspecified"
    ET.SubElement(myObject, "truncated").text = "0"
    ET.SubElement(myObject, "difficult").text = "0"
    bndbox = ET.SubElement(myObject, "bndbox")

    # bndbox child nodes
    ET.SubElement(bndbox, "xmin").text = str(placement_coords[0])
    ET.SubElement(bndbox, "ymin").text = str(placement_coords[1])
    ET.SubElement(bndbox, "xmax").text = str(placement_coords[0] + threat_width)
    ET.SubElement(bndbox, "ymax").text = str(placement_coords[1] + threat_height)

    return xml_annotation

## Check location and size of bounding box

In [ ]:
def pass_size_test(w,h):
    """
    Function evaluates if size of bounding box is ok
    
    Input:
        w, h: width and height of proposed box in px
    
    Output:
        pass_test (bool): True/False
    """
    return (MIN_BBOX_SIZE<=w<=MAX_BBOX_SIZE and MIN_BBOX_SIZE<=h<=MAX_BBOX_SIZE)

## Helper Functions - Cropping Image

In [ ]:
def create_one_crop(x, y, w, h, curr_img, img_name_no_ext, cropped_dir_full, j):
    """
    Takes in bbox coords and image, crops image to the specified dimensions and save it in output.
    
    Input:
        1. x,y,w,h: bbox coords
        2. curr_img (cv2 object): Current image to crop from
        3. img_name_no_ext (str): Image name, for generating of output filename
        4. cropped_dir_full (str): where to save
        5. j (int): Current index of bounding box, for generating of output filename
        
    Returns: Nil
    
    Output:
        1. png file: Saved to output directory.
    """
    # Crop image
    crop_img = curr_img[y:y+h, x:x+w]
    
    # Choose unique filename
    filename = f"{img_name_no_ext}_{j}.png"
    
    # Save image
    cv2.imwrite(os.path.join(cropped_dir_full, filename), crop_img)

## Helper function - generate date-time unique folder name

In [ ]:
def generate_unique_name():
    """
    Generates unique folder name according to current date and time
    
    Returns:
        1. unique_name (str): current date time
    """
    curr_datetime = datetime.now().strftime("%Y_%m_%d-%H.%M.%S")
    
    return curr_datetime

## Helper function - Cropping image with manual XML labels

In [ ]:
def crop_from_manual_XML():
    """
    Given a PascalVOC XML annotation file and the asociated image, this script 
    automatically crops out the objects enclosed within the bounding boxes and saves
    them as standalone images.
    """
    xml_folder_dir = os.path.join(root, XML_LABELS_FOLDER)
    xml_folder_name = os.path.basename(os.path.normpath(xml_folder_dir))
    crop_images_output_dir = os.path.join(root, output_dir, 'cropped_samples', f'manualBoxes_{xml_folder_name}')
    
    # Create new crop folder in output
    if not os.path.exists(crop_images_output_dir):
        os.mkdir(crop_images_output_dir)

    # Get full list of XML files
    xml_list = [name for name in os.listdir(xml_folder_dir) if os.path.isfile(os.path.join(xml_folder_dir, name))]
    
    for i, xml_file in enumerate(xml_list):
        # Get name of XML file
        xml_filename_noext = os.path.splitext(xml_file)[0]
        
        # Read XML file
        tree = ET.parse(os.path.join(xml_folder_dir, xml_file))
        my_root = tree.getroot()
        
        # Open relevant image from main dataset
        curr_img = cv2.imread(os.path.join(root, image_dir, f'{xml_filename_noext}.png'))
        
        # Extract bbox coords
        for j, member in enumerate(my_root.findall('object')):
            
            values = (int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )

            # Crop image
            crop_img = curr_img[values[1]:values[3], values[0]:values[2]]

            # Choose unique filename
            filename = f"{xml_filename_noext}_{j}.png"

            # Save image
            cv2.imwrite(os.path.join(crop_images_output_dir, filename), crop_img)
            
        # Counter
        if (i+1)%PRINT_FREQ==0 or i==(len(xml_list)-1):
            print(f"{i+1}/{len(xml_list)} processed!")
            
    print("Success! Images cropped. See 'cropped_samples' folder")